Текущая работа:
- Прихожу к мнению, что лучше иметь свой полный датасет, и скорее проверять обновление информации с сайта открытых данных, чем использовать напрямую. Логика обновления -- сравнивать ячейки совпадающие в моем датасете с ячейками предпоследней версии открытых данных с новыми ячейками --- если что-то поменялось - предлагать поправить
- Считывание информации из Google Sheet
- Продумать структуру вывода. Пока предполагается: для каждой категории составление списка музеев в виде таблицы - oid - ожидаемая стоимость билета на N дней вперед
- Нужна функция преобразования текста времени работы в конкретные дни недели и интервалы времени, описан ли санитарный день
- Преобразование записей в конкретные дни календаря
- Для каждой категории мы уже наперед знаем, что в ней не меняется-можем заранее генерировать а) значение для особых дней, б) для всех прочих дней, в) главное-учитывать кто открыт и когда у него санитарный день-, 
- Пользователь может сортировать объекты по стоимости/по расположению/рейтингу TripAdvisor (оттуда же можно забирать отзывы)
- С помощью pymorphy оформить полный вывод данных о месте 
- Филиалы Петропавловской крепости - учтены ли - проверить через ссылку сайта https://www.spbmuseum.ru/themuseum/visitors/cost.php
- Не учитывается Коллонада собора

# Сбор открытых данных об основных музеях Петербурга

Передача запроса для поиска открытых данных о музеях Петербурга. Для музеев набор данных имеет [id=123](https://data.gov.spb.ru/opendata/7842489089-museums/). Близкие датасеты 125 и 128. Страница документации https://data.gov.spb.ru/developers/. На сайте есть возможность обновлений, но для надежности, проверка данных происходит по номеру последней версии. Идентификационные номера близких датасетов.

- 123 - Основные музеи города (государственные, федеральные, коммерческие)
- 125 - Выставочные залы
- 128 - Мосты, парки, памятники, природные объекты
- 27  - Объектно-адресная система Санкт-Петербурга

In [1]:
import json
import urllib.request
import pandas

from urllib.error import HTTPError

# Считывание токенов для авторизации в различных сервисах
with open("creds.json", "r") as read_file:
    tokens = json.load(read_file)
    
# Начало url для общения с API открытых данных
# по ссылке без суффиксов передаются описания в виде словарей всех доступных датасетов
datagov_url = 'http://data.gov.spb.ru/api/v1/datasets'
dataset_id = 123

datagov_headers = {"Authorization" : "Token {}".format(tokens['datagov_token'])}

In [2]:
def get_dataset_info(url:str):
    '''Собирает информацию по GET-запросу'''
    req = urllib.request.Request(url, data=None, headers=datagov_headers)
    with urllib.request.urlopen(req) as response:
        html = response.read().decode('utf-8')
    return html


def data_str_preprocessing(url:str):
    '''Преобразование строки данных, полученной по GET-запросу, в словарь'''    
    data_str = get_dataset_info(url)
    data_str = data_str.replace('false', 'False')
    data_str = data_str.replace('true', 'True')
    data_str = data_str.replace('null', 'None')
    return eval(data_str)


def dataset_description(dataset_id:int):
    '''Возвращает словарь формата название поля: описание'''
    url = '{0}/{1}/versions/latest/'.format(datagov_url, dataset_id)
    description = data_str_preprocessing(url)
    columns = {item['title']:item['description'] for item in description['structure']}
    return columns

# Создаем датафрейм

Запустив `dataset_description(dataset_id=dataset_id)`, определелили, что часть полей исходного датасета не представляют интереса для рассмотрения (известна страна нахождения, не требуются ОГРН и ИНН). Для однозначной идентификации используем поле `oid` (регистрационный номер объекта). 

In [3]:
all_columns = dataset_description(dataset_id=dataset_id).keys()
not_important_cols = {'country', 'ogrn', 'inn'}

df = pandas.DataFrame(columns=all_columns)

def get_dataset_page(df, page_num):
    '''Считывает заданную страницу наборад данных'''
    url = '{0}/{1}/versions/latest/data/?page={2}&per_page={3}'.format(datagov_url, dataset_id, page_num, 100)
    page = data_str_preprocessing(url)
    for line in page:
        df = df.append(line['row'], ignore_index=True)
    return df


def get_dataset(df):
    '''Возвращает датафрейм, содержащий все поля, за исключением идентичных'''
    page_num = 1
    while True:
        try:
            df = get_dataset_page(df, page_num)
            page_num += 1
        except HTTPError as e:
            break
    df = df.set_index('oid')
    print('Сохранен список музеев из {} записей.'.format(len(df)))
    df.drop(not_important_cols, axis=1, inplace=True) # выкидываем неинтересные столбцы
    return df

df = get_dataset(df)

Сохранен список музеев из 255 записей.


# Выделение интервалов рабочего времени (в процессе)
Необходимо определить часы работы для каждого дня, если день выходной или санитарный - пишем `off`. Сформулируем пока словами правила поиска рабочих дней, рабочего времени, нерабочих дней:
- Если строка начинается с временного интервала, значит все следующие описательные строки - исключения из этого правила (можно описать правило, а потом заменить остальные ячейки), аналогично для слова `ежедневно`
- Сначала идут даты, потом время для этих дат
- Если строка начинается с дней недели, а дальше следует время, то это и есть время и рабочие часы
- Исключения пишутся в конце, их можно определить по словам: `кроме`, `закрыт`, `выходной`, `санитарный`

In [47]:
short_ru_weekdays = ('пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс')
long_ru_weekdays = ('понедельник',
                    'вторник',
                    'среда',
                    'четверг',
                    'пятница',
                    'суббота',
                    'воскресенье')


Данные требуют приведения к единообразию. Так, различным образом оформлены поля
- `name` - нормальным образом и заглавными буквами 
- `phone` - идентичное представление номера телефона

Необходимо проверить полноту имеющейся информации.
Дополнить информацию можно следующими разделами:
- Ссылка на страницу с указанием цены билетов
- Бесплатные дни для посещения музея
- Ссылку на страницу с изображением места

Дополнительные данные хранятся в [Google Sheet](https://docs.google.com/spreadsheets/d/11AgqYCxS1krnRQRWzhJmtFVpHA01OkjYAwYtARFHiV0/edit?usp=sharing).

Поля адреса можно использовать для краткого описания с помощью pymorphy, в виде строк
"Музей {name} расположен в {subdistrict} {district} в доме {home} {street} / если не Петербург - города {town}/. Музей находится в {} мин. ходьбы от метро {}/ Часы работы {work time}, Телефон/ы для связи: {phone} e-mail {email} / Если есть сайт / Проверяйте информацию на сайте {www}"

In [51]:
df['obj_history'][4]

'«Императорское воспитательное общество благородных девиц» было учреждено 5 мая 1764 года указом Екатерины II по инициативе И.И. Бецкого и стало первым в России женским учебным заведением. Первоначально оно предназначалось только для дочерей дворянской знати, но уже год спустя было открыто отделение «для мещанских девиц», т.е. недворянских сословий, кроме крепостных крестьян. В шестилетнем возрасте девочки поступали на обучение, которое продолжалось двенадцать лет. Сначала воспитательное общество располагалось в Новодевичьем монастыре, в 1800-е гг. для института было построено специальное здание, учебное заведение стало именоваться Смольным институтом благородных девиц. В октябре 1917 года институт во главе с княгиней В.В. Голицыной переехал в Новочеркасск. В здании Смольного института стали располагаться Петроградский Совет рабочих и крестьянских депутатов и ЦИК Советов, а позднее Центральный и Петроградский комитеты большевиков. До 1991 года здесь размещались Областной и Городской ко

# Дополнительная информация

Бесплатный билет (кроме билетов для детей до 18 лет) выдается при предъявлении соответствующих документов, подтверждающих право на него. Документы должны быть действующими (продленными) на дату получения билета. В студенческих документах (студ. билеты или ISIC) должны быть: фотография, дата окончания действия, учебное заведение.